# Find proteins outside the 8q gain event (trans) that are commonly different between samples with and without the event

## Setup

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
ttest_results_file = "8q_transeffects.tsv"

ttest_results = pd.\
read_csv(ttest_results_file, sep="\t").\
rename(columns={"Name": "protein"}).\
set_index("protein")

In [3]:
ttest_results

,lscc_Database_ID,luad_Database_ID,ovarian_Database_ID,colon_pvalue,hnscc_pvalue,lscc_pvalue,luad_pvalue,ovarian_pvalue,colon_diff,hnscc_diff,lscc_diff,luad_diff,ovarian_diff
protein,,,,,,,,,,,,,
A1BG,NP_570602.2,NP_570602.2,NP_570602,0.183228,0.699736,0.794905,0.980069,0.910369,-0.363979,-0.075722,0.147324,0.018228,-0.096252
A2M,NP_000005.2|NP_001334353.1|NP_001334354.1,NP_000005.2|NP_001334353.1|NP_001334354.1|K4JD...,NP_000005,0.569279,0.833774,0.868751,0.814395,0.987964,-0.153202,-0.073016,-0.139115,0.181837,0.022644
AAAS,NP_056480.1|NP_001166937.1,NP_056480.1|NP_001166937.1,NP_056480,0.609299,0.881926,0.772112,0.350913,0.873217,0.089587,0.020537,-0.075273,0.208068,0.045529
AACS,NP_076417.2|NP_001306769.1|NP_001306768.1,NP_076417.2|NP_001306769.1|NP_001306768.1,NP_076417,0.908476,0.749539,0.915220,0.673566,0.926280,0.022990,0.088849,-0.092006,0.326426,-0.068586
AAGAB,NP_078942.3|NP_001258814.1,NP_078942.3|NP_001258814.1,NP_078942,0.322962,0.881614,0.839262,0.973398,0.884892,-0.144167,0.034488,-0.091437,-0.020575,-0.064130
AAK1,NP_055726.3,NaN,NP_055726,0.213117,0.447466,0.396367,NaN,0.774287,-0.110944,-0.078541,-0.261571,NaN,-0.111739
AAMDC,NP_001303889.1|NP_001350493.1,NP_001303889.1|NP_001350493.1|NP_001303886.1|N...,NP_078960,0.295965,0.935797,0.653037,0.758739,0.865407,0.183176,-0.026586,-0.278158,-0.227490,-0.132402
AAMP,NP_001289474.1|NP_001078.2,NP_001289474.1|NP_001078.2,NP_001078,0.989918,0.879966,0.357651,0.627132,0.794877,-0.002691,-0.034102,-0.404214,-0.200570,-0.112696
AAR2,NP_001258803.1,NP_001258803.1,NP_001258803,0.065394,0.858159,0.337295,0.530154,0.775761,0.193667,0.034607,-0.290468,0.177188,-0.107719


## Reshape the input dataframe
We want to get our table to have these columns:
- cancer_type
- protein
- Database_ID
- change
- p_value

Since some cancer types have database IDs and some don't, we'll slice out and reshape the info for each cancer type individually.

In [4]:
cancer_types = sorted(ttest_results.columns.to_series().str.split("_", n=1, expand=True)[0].unique())

long_results = pd.DataFrame()

for cancer_type in cancer_types:
    cancer_df = ttest_results.\
    loc[:, ttest_results.columns.str.startswith(cancer_type)].\
    dropna(axis="index", how="all").\
    reset_index(drop=False)
    
    # If the cancer type has database IDs, make a separate column that has them.
    # If not, create a column of NaNs (so that the tables all match)
    if f"{cancer_type}_Database_ID" in cancer_df.columns:
        cancer_df = cancer_df.rename(columns={f"{cancer_type}_Database_ID": "Database_ID"})
    else:
        cancer_df = cancer_df.assign(Database_ID=np.nan)
        
    # Rename the pvalue and diff columns to not have the cancer type
    cancer_df = cancer_df.rename(columns={
        f"{cancer_type}_pvalue": "adj_p",
        f"{cancer_type}_diff": "change"
    }).\
    assign(cancer_type=cancer_type)
    
    # Reorder the columns
    cancer_df = cancer_df[["cancer_type", "protein", "Database_ID", "adj_p", "change"]]
    
    # Append to the overall dataframe
    long_results = long_results.append(cancer_df)

# Drop duplicate rows and reset the index
long_results = long_results[~long_results.duplicated(keep=False)].\
reset_index(drop=True)

In [5]:
long_results

,cancer_type,protein,Database_ID,adj_p,change
0,colon,A1BG,NaN,0.183228,-0.363979
1,colon,A2M,NaN,0.569279,-0.153202
2,colon,AAAS,NaN,0.609299,0.089587
3,colon,AACS,NaN,0.908476,0.022990
4,colon,AAGAB,NaN,0.322962,-0.144167
5,colon,AAK1,NaN,0.213117,-0.110944
6,colon,AAMDC,NaN,0.295965,0.183176
7,colon,AAMP,NaN,0.989918,-0.002691
8,colon,AAR2,NaN,0.065394,0.193667
9,colon,AARS2,NaN,0.733199,0.048838


## Select the proteins with a significant change

In [6]:
prots = long_results[long_results["adj_p"] <= 0.05].reset_index(drop=True)

In [8]:
prots.groupby("cancer_type").count()

,protein,Database_ID,adj_p,change
cancer_type,,,,
colon,100,0,100,100
hnscc,1,0,1,1
lscc,6,6,6,6
luad,24,24,24,24


## Find how many cancers each protein was different in

In [7]:
prots_summary = prots.groupby("protein").agg(**{
    "cancers": ("cancer_type", lambda x: x.sort_values().drop_duplicates(keep="first").tolist())
})

prots_summary = prots_summary.\
assign(
    num_cancers=prots_summary["cancers"].apply(len),
    tmp_sort=prots_summary["cancers"].apply(lambda x: "".join(x))
).\
sort_values(by=["num_cancers", "tmp_sort"], ascending=[False, True]).\
drop(columns="tmp_sort")

prots_summary

,cancers,num_cancers
protein,,
UQCC1,"[colon, luad]",2
UQCRH,"[colon, luad]",2
ACAD9,[colon],1
ACSS1,[colon],1
ADNP,[colon],1
AIFM1,[colon],1
AKAP1,[colon],1
ALDH5A1,[colon],1
ANKFY1,[colon],1
